### 1. 正则的几个用法

1. 匹配手机号

In [29]:
import re

phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
result = re.findall(phone_p, 'Myphone is :135-0009-4747 or 188 4786 8856 or 15878963367')
print(result)

area_pattern = re.compile('http://([\w]*)\.adjia\.cn')
result1 = re.match(area_pattern, 'http://yibin.adjia.cn/')
print(result1.group(1))



pattern_shop_count = re.compile('.*-page-([0-9]*)\.')
result = re.match(pattern_shop_count, 'list-htm-fid-1-page-26.html')
print(result.group(1))

['135-0009-4747', '188 4786 8856', '15878963367']
yibin
26


2.贪婪匹配和非贪婪匹配

- 贪婪匹配: *

- 非贪婪匹配(可能为0个)： *?

- 贪婪匹配所有字符:  .*  

- 非贪婪匹配所有字符: .*? 

In [7]:
import re
p1 = re.compile(r'我爱好.*')
result = re.match(p1, '我爱好好好好')
print(result)

p2 = re.compile(r'我爱好.*?')
result = re.match(p2, '我爱好好好好')
print(result)

<_sre.SRE_Match object; span=(0, 6), match='我爱好好好好'>
<_sre.SRE_Match object; span=(0, 3), match='我爱好'>


3.使用捕获组

括号的使用
findall可以用于找只有一个匹配内容的，相当于只有一个括号的情况，但是当有多个括号的时候，使用findall就会比较麻烦

In [8]:
p = re.compile(r'[a-z]+((\d+)-(\d+))[a-z]+')
 
result = re.match(p, 'afs123-456gds')
print(result.group()) # afs123-456gds
print(result.group(1)) # 123-456
print(result.group(2)) # 123
print(result.group(3)) # 456

afs123-456gds
123-456
123
456


4.字符串的替换

In [9]:
import re
 
p = re.compile(r'\d+')
 
result = re.sub(p, '中文', 'tom32jerry456haha')
print(result) # tom中文jerry中文haha

tom中文jerry中文haha


### 2.构建谷歌搜索函数

- 关键点1:如果实现翻墙？在本地使用了小飞机，如何让我们的python请求也经过小飞机？答案是使用代理，小飞机的代理端口是1080
所以只需要将代理指定到本地的1080即可

- 关键点2:仔细观察谷歌搜索时向服务器发送的请求，可以获得最近一年，最近一个月最近一周的结果


In [6]:
import re
import requests 
from bs4 import BeautifulSoup
proxy = {"http":"http://127.0.0.1:1080","https":"https://127.0.0.1:1080"}
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')

def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    tbs = '&tbs=qdr:y'
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
#     url = 'https://www.google.com/search?q=' + q + '&start=10&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get,proxies = proxy,verify=False)

    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.find('div', id='resultStats'))
    output = []
#     for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
#         url = searchWrapper.find('a')["href"] 
#         text = searchWrapper.find('a').text.strip()
#         result = {'text': text, 'url': url}
#         output.append(result)
    
    for content_p in soup.find_all('span', {'class':'st'}):
        text = content_p.text
        result = {'text': text}
        output.append(result)
#         print(text)
        resultm = re.findall(phone_p, text)
        print(resultm)
        print("\n")
    return output

(google("hello"))

E:\Users\binj\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
E:\Users\binj\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


None


[]

In [4]:
#coding:utf-8
import requests
import json
import time
import re

headers = {
    'Host':"map.baidu.com",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0"}

def citys():
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=s&da_src=searchBox.button&wd=%E6%B1%BD%E8%BD%A6%E7%BE%8E%E5%AE%B9%E5%BA%97&c=1&src=0&wd2=&sug=0&l=5&b=(7002451.220000001,1994587.88;19470675.22,7343963.88)&from=webmap&biz_forward={%22scaler%22:1,%22styles%22:%22pl%22}&sug_forward=&tn=B_NORMAL_MAP&nn=0&u_loc=12736591.152491,3547888.166124&ie=utf-8&t=1459951988807',headers=headers).text
    f=open('city_ids.txt','a')
    data=json.loads(html)
    for item in data['content']:
        #for city in item['city']:
            f.write(str(item)+'\n')
    f.close()

def get_infor(keyword,code,page):
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=con&from=webmap&c='+str(code)+'&wd='+keyword+'&wd2=&pn='+str(page)+'&nn='+str(page*10)+'&db=0&sug=0&addr=0&&da_src=pcmappg.poi.page&on_gel=1&src=7&gr=3&l=12&tn=B_NORMAL_MAP&u_loc=12736591.152491,3547888.166124&ie=utf-8',headers=headers).text
    data=json.loads(html)['content']
    return data


def main():
    keys=['深圳 横岗 广告']
    for keyword in keys:
        f=open(keyword+'_tels.txt','a')
        code=340
        page=1
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
                f.write(str(item['area_name'])+',    ')
                f.write(str(item['name'])+',    ')
                f.write(str(item['addr'])+',    ')
                f.write(str(item['ext']['detail_info']['phone'])+'\n')
                # print(str(item['area_name'])+'\n')

            page+=1
            print(code,page)
            time.sleep(1)
        f.close()
main()

340 2
340 3
340 4
340 5
340 6
340 7
340 8
340 9


In [8]:
def getWeChat():
    keys=['龙岗中心城 广告']
    for keyword in keys:
        code=340
        page=1
        f=open(keyword+'_tels.txt','a')
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
#                 print(str(item['name'])+' ')
                content = '\"'+str(item['area_name'])+'\" ' + '\"' + str(item['name']) + '\"' +' \"微信\"'
                print(content)
                google(content)

            page+=1
            print(code,page)
            time.sleep(1)         
        f.close()
getWeChat()

In [4]:
import requests
import json
import time
import re
from bs4 import BeautifulSoup
import xlwt

# get all phone number from： http://sz.adjia.cn/  ，
phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }



def get_phone(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"lxml")
    phone_num = soup.find('div', {'class':'telphone'}).text
    resultm = re.findall(phone_p, phone_num)
    return resultm[0]


def page_h4(index_start, index_end):
    # 深圳
    url_head = 'http://sz.adjia.cn/f/'
    # 北京
    url_head = 'http://sh.adjia.cn/f/'
    excel_book = xlwt.Workbook(encoding='ascii')
    worksheet = excel_book.add_sheet('My Worksheet')
    phone_count = 0
    for page_num in range(index_start,index_end):
        page_url = url_head + 'list-htm-fid-1-page-{}.html'.format(page_num)
#         print(page_url)
        r = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(r.text,"lxml")
        for content_list in soup.find_all('div', {'class':'ListInfo'}):
            children = content_list.find_all('h4')    
            for child in children:
                print(child.text)
                worksheet.write(phone_count, 0, label = child.text)
                href = url_head + child.a['href']
#                 print(get_phone(href))
                worksheet.write(phone_count, 1, label = get_phone(href))
                phone_count+=1
    excel_book.save('Excel_Workbook_{}.xls'.format(index_start))

# for i in range(1,10):
#     start = 0
#     if i== 1:
#         page_h4(1,10)
#     else:
#         page_h4((i-1)*10,i*10)
        

In [38]:
def get_area_shop_count(url):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text,"lxml")
    show_page = soup.find('div', {'class':'ShowPage'})
    shop_count_index = len(show_page.find_all('a'))
    shop_count = show_page.find_all('a')[shop_count_index-2]['href']
    pattern_shop_count = re.compile('.*-page-([0-9]*)\.')
    result = re.match(pattern_shop_count, shop_count)
    num = int(result.group(1))
    return num
    
# get_area_shop_count('http://xuancheng.adjia.cn/f/list-htm-fid-1.html')

def get_area_name():
    start_url = 'http://www.adjia.cn/'
    area_pattern = re.compile('http://([\w]*)\.adjia\.cn')
    area_count = 0
    r = requests.get(start_url, headers=headers)
    soup = BeautifulSoup(r.text,"lxml")
    for content_list in soup.find_all('div', {'class':'area_citys'}):
        children = content_list.find_all('a')    
        for child in children:
            area_count+=1
            area_name = re.match(area_pattern, child['href'])
            area_name = (area_name.group(1))
            print(area_name)
            print(child.text)
            shop_url_head = 'http://{}.adjia.cn/f/'.format(area_name)
            shop_url = shop_url_head + 'list-htm-fid-1.html'.format(area_name)
#             shop_count = get_area_shop_count(shop_url)
#             print(shop_count)
    print('total count {}'.format(area_count))


get_area_name()

anqing
安庆
anshun
安顺
anyang
安阳
anshan
鞍山
alashan
阿拉善
ankang
安康
aba
阿坝
ali
阿里
akesu
阿克苏
aletai
阿勒泰
bj
北京
bangbu
蚌埠
baiyin
白银
beihai
北海
baise
百色
bijie
毕节
baoding
保定
baishan
白山
baicheng
白城
benxi
本溪
baotou
包头
bayannaoer
巴彦淖尔
binzhou
滨州
baoji
宝鸡
bazhong
巴中
bayinguoleng
巴音郭楞
boertala
博尔塔拉
baoshan
保山
cq
重庆
chaozhou
潮州
chuzhou
滁州
chaohu
巢湖
chizhou
池州
chongzuo
崇左
cangzhou
沧州
chengde
承德
changsha
长沙
changde
常德
chenzhou
郴州
changchun
长春
changzhou
常州
chaoyang
朝阳
chifeng
赤峰
changzhi
长治
cd
成都
changdu
昌都
changji
昌吉
chuxiong
楚雄
dong
东莞
dingxi
定西
daqing
大庆
daxinganling
大兴安岭
dalian
大连
dandong
丹东
dongying
东营
dezhou
德州
datong
大同
deyang
德阳
dazhou
达州
dali
大理
dehong
德宏
diqing
迪庆
ezhou
鄂州
enshi
恩施
eerduosi
鄂尔多斯
foshan
佛山
fuyang
阜阳
fuzhou
福州
fangchenggang
防城港
fuzhou
抚州
fushun
抚顺
fuxin
阜新
gz
广州
gannan
甘南
guilin
桂林
guigang
贵港
guiyang
贵阳
ganzhou
赣州
guyuan
固原
guoluo
果洛
guangan
广安
guangyuan
广元
ganzi
甘孜
huizhou
惠州
heyuan
河源
hz
杭州
huzhou
湖州
hefei
合肥
huainan
淮南
huaibei
淮北
huangshan
黄山
hechi
河池
hezhou
贺州
haikou
海口
handan
